### Hypothesis 5: Stress tests show PD sensitivity to leverage, asset volatility, and interest rates.

Basic Idea:
- choose firms
- find baseline pd
- choose stress scenarios
    - 10% change in debt
    - 20% change in volatility
    - 1% change in interest rate
    - different combinations of stress scenarios
- calculate pd in stress scenarios
- calculate pd delta for each stress scenario
- use metrics like mean delta pd, % delta pd, or correlation to see sensitivity levels

In [24]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import root

In [25]:
import os
print(os.getcwd())

/Users/brockwilliams/Desktop/MF728_Project/hypothesis-5


In [27]:
import os
print(os.listdir('../data'))

['data_processing.ipynb', 'api_access.ipynb', 'TreasuryRaw.csv', 'README.md', 'raw_data.csv', 'clean_data.csv', '.ipynb_checkpoints']


In [28]:
def compute_merton_pd(E, sigma_E, D, r, T=1.0, verbose=False):    

    def equations(vars):
        V, sigma_V = vars
        if V <= 0 or sigma_V <= 0:
            return 1e10, 1e10  # large penalty for invalid values
        d1 = (np.log(V / D) + (r + 0.5 * sigma_V**2) * T) / (sigma_V * np.sqrt(T))
        d2 = d1 - sigma_V * np.sqrt(T)
        eq1 = V * norm.cdf(d1) - D * np.exp(-r * T) * norm.cdf(d2) - E
        eq2 = (V / E) * norm.cdf(d1) * sigma_V - sigma_E
        return [eq1, eq2]

    V0 = E + D
    sigma_V0 = sigma_E
    result = root(equations, [V0, sigma_V0], method='hybr')

    if result.success:
        V_opt, sigma_V_opt = result.x
        d2 = (np.log(V_opt / D) + (r - 0.5 * sigma_V_opt ** 2) * T) / (sigma_V_opt * np.sqrt(T))
        pd = norm.cdf(-d2)
        return pd
    else:
        if verbose:
            print("Failure for firm:", E, sigma_E, D, r)
            print("Message:", result.message)
        return np.nan

In [32]:
df= pd.read_csv('../data/clean_data.csv')
df = df.sample(n=10000, random_state=42)
df['merton_pd'] = df.apply(
    lambda row: compute_merton_pd(
        E=row['market_cap'],
        sigma_E=row['equity_volatility'],
        D=row['total_debt'] * 0.9,
        r=row['rf'] / 100,  # Convert percentage to decimal
        verbose=True
    ),
    axis=1
)
print(df)

/var/folders/59/hb0z0gw54v346g6175w3ns9r0000gp/T/ipykernel_23568/2908855909.py:7: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (np.log(V / D) + (r + 0.5 * sigma_V**2) * T) / (sigma_V * np.sqrt(T))
/var/folders/59/hb0z0gw54v346g6175w3ns9r0000gp/T/ipykernel_23568/2908855909.py:19: RuntimeWarning: divide by zero encountered in scalar divide
  d2 = (np.log(V_opt / D) + (r - 0.5 * sigma_V_opt ** 2) * T) / (sigma_V_opt * np.sqrt(T))


Failure for firm: 65878485199.99999 0.2347157656572713 316177200000.0 0.020099999999999996
Message: The number of calls to function has reached maxfev = 600.
Failure for firm: 4167692700.0 1.6685693032385236 119135700000.0 0.0393
Message: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
Failure for firm: 42963268870.0 1.3493884916721397 271382850000.0 0.0349
Message: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
Failure for firm: 74717914560.0 0.3267410703739833 400612500000.0 0.0256
Message: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
Failure for firm: 18146968200.0 1.2543565596693236 526280850000.0 0.027000000000000003
Message: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
Failure for firm: 20342232000.0 1.6043449181714664 467555400000.0 0.0371


In [33]:
df['pd_valid'] = df['merton_pd'].notna()
df.to_csv("merton_model_output.csv", index=False)

In [34]:
df['pd_valid'].value_counts()

pd_valid
True     9962
False      38
Name: count, dtype: int64

In [21]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

print(df[df['merton_pd'].isna() == True])
(df[df['merton_pd'].isna() == True]).to_csv("nans.csv", index=False)

pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.max_colwidth')

              date  permno  tic                          conm        PRC  \
277177  2015-12-29   69032   MS                MORGAN STANLEY   32.55000   
275961  2011-03-01   69032   MS                MORGAN STANLEY   28.82000   
23805   2008-10-29   66800  AIG  AMERICAN INTERNATIONAL GROUP    1.55000   
275598  2009-09-21   69032   MS                MORGAN STANLEY   31.61000   
55567   2010-08-17   59408  BAC          BANK OF AMERICA CORP   13.21000   
181129  2010-09-23   86868   GS       GOLDMAN SACHS GROUP INC  144.91000   
90393   2009-01-26   70519    C                 CITIGROUP INC    3.33000   
90480   2009-06-01   70519    C                 CITIGROUP INC    3.69000   
55243   2009-05-05   59408  BAC          BANK OF AMERICA CORP   10.84000   
275444  2009-02-10   69032   MS                MORGAN STANLEY   20.79000   
56289   2013-07-01   59408  BAC          BANK OF AMERICA CORP   12.93000   
91137   2012-01-06   70519    C                 CITIGROUP INC   28.55000   
90504   2009

In [61]:
display(df)

,date,permno,tic,conm,PRC,atq,dlcq,dlttq,SHROUT,market_cap,total_debt,leverage,log_return,equity_volatility,rf,merton_pd,pd_valid
0,2006-01-03,14593,AAPL,APPLE INC,74.75,1.418100e+10,0.000000e+00,0.000000e+00,845617.0,6.320987e+10,0.000000e+00,0.000000,0.039012,0.790549,4.37,0.000000e+00,True
1,2006-01-04,14593,AAPL,APPLE INC,74.97,1.418100e+10,0.000000e+00,0.000000e+00,845617.0,6.339591e+10,0.000000e+00,0.000000,0.002939,0.790494,4.36,0.000000e+00,True
2,2006-01-05,14593,AAPL,APPLE INC,74.38,1.418100e+10,0.000000e+00,0.000000e+00,845617.0,6.289699e+10,0.000000e+00,0.000000,-0.007901,0.790498,4.36,0.000000e+00,True
3,2006-01-06,14593,AAPL,APPLE INC,76.30,1.418100e+10,0.000000e+00,0.000000e+00,845617.0,6.452058e+10,0.000000e+00,0.000000,0.025486,0.790891,4.38,0.000000e+00,True
4,2006-01-09,14593,AAPL,APPLE INC,76.05,1.418100e+10,0.000000e+00,0.000000e+00,845617.0,6.430917e+10,0.000000e+00,0.000000,-0.003282,0.787805,4.38,0.000000e+00,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441045,2024-12-24,11850,XOM,EXXON MOBIL CORP,106.40,4.619160e+11,5.632000e+09,3.691800e+10,4395095.0,4.676381e+11,2.409100e+10,0.092116,0.000940,0.192976,4.59,3.915704e-80,True
441046,2024-12-26,11850,XOM,EXXON MOBIL CORP,106.49,4.619160e+11,5.632000e+09,3.691800e+10,4395095.0,4.680337e+11,2.409100e+10,0.092116,0.000846,0.192966,4.58,3.582220e-80,True
441047,2024-12-27,11850,XOM,EXXON MOBIL CORP,106.48,4.619160e+11,5.632000e+09,3.691800e+10,4395095.0,4.679897e+11,2.409100e+10,0.092116,-0.000094,0.192904,4.62,3.098549e-80,True
441048,2024-12-30,11850,XOM,EXXON MOBIL CORP,105.76,4.619160e+11,5.632000e+09,3.691800e+10,4395095.0,4.648252e+11,2.409100e+10,0.092116,-0.006785,0.192463,4.55,2.732413e-80,True


In [22]:
nans= pd.read_csv('nans.csv')
print(nans['leverage'].describe())
print()
print(nans['equity_volatility'].describe())

count    36.000000
mean      0.432765
std       0.144222
min       0.172107
25%       0.342929
50%       0.387375
75%       0.579532
max       0.612120
Name: leverage, dtype: float64

count    36.000000
mean      1.270800
std       0.857868
min       0.197453
25%       0.327128
50%       1.340437
75%       1.604376
max       3.507883
Name: equity_volatility, dtype: float64


In [23]:
print(df['leverage'].describe())
print()
print(df['equity_volatility'].describe())

count    10000.000000
mean         0.264841
std          0.165679
min          0.000000
25%          0.149034
50%          0.239428
75%          0.359652
max          1.039717
Name: leverage, dtype: float64

count    10000.000000
mean         2.143514
std         14.499943
min          0.094552
25%          0.192288
50%          0.245272
75%          0.332461
max        116.563323
Name: equity_volatility, dtype: float64


#### 10% Change in Debt

In [36]:
df= pd.read_csv('../data/clean_data.csv')
df['merton_pd'] = df.apply(
    lambda row: compute_merton_pd(
        E=row['market_cap'],
        sigma_E=row['equity_volatility'],
        D=row['total_debt']*0.9,
        r=row['rf'] / 100,  # Convert percentage to decimal
        verbose=True
    ),
    axis=1
)
print(df)

/var/folders/59/hb0z0gw54v346g6175w3ns9r0000gp/T/ipykernel_23568/2908855909.py:7: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (np.log(V / D) + (r + 0.5 * sigma_V**2) * T) / (sigma_V * np.sqrt(T))
/var/folders/59/hb0z0gw54v346g6175w3ns9r0000gp/T/ipykernel_23568/2908855909.py:19: RuntimeWarning: divide by zero encountered in scalar divide
  d2 = (np.log(V_opt / D) + (r - 0.5 * sigma_V_opt ** 2) * T) / (sigma_V_opt * np.sqrt(T))


Failure for firm: 12798849840.0 1.184655266718994 120536550000.0 0.0347
Message: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
Failure for firm: 10083127500.0 1.2063649383315511 120536550000.0 0.0348
Message: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
Failure for firm: 5512109699.999999 1.3433913552168046 120536550000.0 0.0341
Message: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
Failure for firm: 7232963460.0 1.3732017429642132 120536550000.0 0.0354
Message: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
Failure for firm: 10352010900.0 1.4225119557038712 120536550000.0 0.0378
Message: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
Failure for firm: 12691296479.999998 1.43863099251560

In [37]:
df['pd_valid'] = df['merton_pd'].notna()
df.to_csv("merton_model_output.csv", index=False)

In [38]:
df['pd_valid'].value_counts()

pd_valid
True     439265
False      1785
Name: count, dtype: int64

In [39]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

print(df[df['merton_pd'].isna() == True])
(df[df['merton_pd'].isna() == True]).to_csv("nans.csv", index=False)

pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.max_colwidth')

              date  permno  tic                          conm        PRC  \
23773   2008-09-15   66800  AIG  AMERICAN INTERNATIONAL GROUP    4.76000   
23774   2008-09-16   66800  AIG  AMERICAN INTERNATIONAL GROUP    3.75000   
23775   2008-09-17   66800  AIG  AMERICAN INTERNATIONAL GROUP    2.05000   
23776   2008-09-18   66800  AIG  AMERICAN INTERNATIONAL GROUP    2.69000   
23777   2008-09-19   66800  AIG  AMERICAN INTERNATIONAL GROUP    3.85000   
23778   2008-09-22   66800  AIG  AMERICAN INTERNATIONAL GROUP    4.72000   
23779   2008-09-23   66800  AIG  AMERICAN INTERNATIONAL GROUP    5.00000   
23780   2008-09-24   66800  AIG  AMERICAN INTERNATIONAL GROUP    3.31000   
23781   2008-09-25   66800  AIG  AMERICAN INTERNATIONAL GROUP    3.02000   
23782   2008-09-26   66800  AIG  AMERICAN INTERNATIONAL GROUP    3.15000   
23783   2008-09-29   66800  AIG  AMERICAN INTERNATIONAL GROUP    2.50000   
23784   2008-09-30   66800  AIG  AMERICAN INTERNATIONAL GROUP    3.33000   
23785   2008

In [62]:
display(df)

,date,permno,tic,conm,PRC,atq,dlcq,dlttq,SHROUT,market_cap,total_debt,leverage,log_return,equity_volatility,rf,merton_pd,pd_valid
0,2006-01-03,14593,AAPL,APPLE INC,74.75,1.418100e+10,0.000000e+00,0.000000e+00,845617.0,6.320987e+10,0.000000e+00,0.000000,0.039012,0.790549,4.37,0.000000e+00,True
1,2006-01-04,14593,AAPL,APPLE INC,74.97,1.418100e+10,0.000000e+00,0.000000e+00,845617.0,6.339591e+10,0.000000e+00,0.000000,0.002939,0.790494,4.36,0.000000e+00,True
2,2006-01-05,14593,AAPL,APPLE INC,74.38,1.418100e+10,0.000000e+00,0.000000e+00,845617.0,6.289699e+10,0.000000e+00,0.000000,-0.007901,0.790498,4.36,0.000000e+00,True
3,2006-01-06,14593,AAPL,APPLE INC,76.30,1.418100e+10,0.000000e+00,0.000000e+00,845617.0,6.452058e+10,0.000000e+00,0.000000,0.025486,0.790891,4.38,0.000000e+00,True
4,2006-01-09,14593,AAPL,APPLE INC,76.05,1.418100e+10,0.000000e+00,0.000000e+00,845617.0,6.430917e+10,0.000000e+00,0.000000,-0.003282,0.787805,4.38,0.000000e+00,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441045,2024-12-24,11850,XOM,EXXON MOBIL CORP,106.40,4.619160e+11,5.632000e+09,3.691800e+10,4395095.0,4.676381e+11,2.409100e+10,0.092116,0.000940,0.192976,4.59,3.915704e-80,True
441046,2024-12-26,11850,XOM,EXXON MOBIL CORP,106.49,4.619160e+11,5.632000e+09,3.691800e+10,4395095.0,4.680337e+11,2.409100e+10,0.092116,0.000846,0.192966,4.58,3.582220e-80,True
441047,2024-12-27,11850,XOM,EXXON MOBIL CORP,106.48,4.619160e+11,5.632000e+09,3.691800e+10,4395095.0,4.679897e+11,2.409100e+10,0.092116,-0.000094,0.192904,4.62,3.098549e-80,True
441048,2024-12-30,11850,XOM,EXXON MOBIL CORP,105.76,4.619160e+11,5.632000e+09,3.691800e+10,4395095.0,4.648252e+11,2.409100e+10,0.092116,-0.006785,0.192463,4.55,2.732413e-80,True


In [40]:
nans= pd.read_csv('nans.csv')
print(nans['leverage'].describe())
print()
print(nans['equity_volatility'].describe())

count    1785.000000
mean        0.409438
std         0.154642
min         0.133314
25%         0.257652
50%         0.374395
75%         0.569141
max         0.651163
Name: leverage, dtype: float64

count    1785.000000
mean        1.335189
std         0.864648
min         0.184574
25%         0.390830
50%         1.389906
75%         1.747623
max         3.508130
Name: equity_volatility, dtype: float64


In [41]:
print(df['leverage'].describe())
print()
print(df['equity_volatility'].describe())

count    441050.000000
mean          0.263176
std           0.163073
min           0.000000
25%           0.149472
50%           0.240519
75%           0.358133
max           1.039717
Name: leverage, dtype: float64

count    441050.000000
mean          2.226904
std          14.809782
min           0.090416
25%           0.189931
50%           0.243494
75%           0.330184
max         116.571964
Name: equity_volatility, dtype: float64


#### 10% Change in Volatility

In [42]:
df= pd.read_csv('../data/clean_data.csv')
df['merton_pd'] = df.apply(
    lambda row: compute_merton_pd(
        E=row['market_cap'],
        sigma_E=row['equity_volatility'] * 0.9,
        D=row['total_debt'],
        r=row['rf'] / 100,  # Convert percentage to decimal
        verbose=True
    ),
    axis=1
)
print(df)

/var/folders/59/hb0z0gw54v346g6175w3ns9r0000gp/T/ipykernel_23568/2908855909.py:7: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (np.log(V / D) + (r + 0.5 * sigma_V**2) * T) / (sigma_V * np.sqrt(T))
/var/folders/59/hb0z0gw54v346g6175w3ns9r0000gp/T/ipykernel_23568/2908855909.py:19: RuntimeWarning: divide by zero encountered in scalar divide
  d2 = (np.log(V_opt / D) + (r - 0.5 * sigma_V_opt ** 2) * T) / (sigma_V_opt * np.sqrt(T))


Failure for firm: 34143950000.0 0.13374030319897778 256050000.0 0.0273
Message: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
Failure for firm: 12798849840.0 1.0661897400470945 133929500000.0 0.0347
Message: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
Failure for firm: 10083127500.0 1.0857284444983961 133929500000.0 0.0348
Message: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
Failure for firm: 7232963460.0 1.235881568667792 133929500000.0 0.0354
Message: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
Failure for firm: 10352010900.0 1.280260760133484 133929500000.0 0.0378
Message: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
Failure for firm: 12691296479.999998 1.2947678932640423 1339

In [43]:
df['pd_valid'] = df['merton_pd'].notna()
df.to_csv("merton_model_output1.csv", index=False)

In [44]:
df['pd_valid'].value_counts()

pd_valid
True     439128
False      1922
Name: count, dtype: int64

In [45]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

print(df[df['merton_pd'].isna() == True])
(df[df['merton_pd'].isna() == True]).to_csv("nans.csv", index=False)

pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.max_colwidth')

              date  permno  tic                          conm        PRC  \
20449   2013-07-05   44644  ADP     AUTOMATIC DATA PROCESSING   70.75000   
23773   2008-09-15   66800  AIG  AMERICAN INTERNATIONAL GROUP    4.76000   
23774   2008-09-16   66800  AIG  AMERICAN INTERNATIONAL GROUP    3.75000   
23776   2008-09-18   66800  AIG  AMERICAN INTERNATIONAL GROUP    2.69000   
23777   2008-09-19   66800  AIG  AMERICAN INTERNATIONAL GROUP    3.85000   
23778   2008-09-22   66800  AIG  AMERICAN INTERNATIONAL GROUP    4.72000   
23779   2008-09-23   66800  AIG  AMERICAN INTERNATIONAL GROUP    5.00000   
23780   2008-09-24   66800  AIG  AMERICAN INTERNATIONAL GROUP    3.31000   
23781   2008-09-25   66800  AIG  AMERICAN INTERNATIONAL GROUP    3.02000   
23782   2008-09-26   66800  AIG  AMERICAN INTERNATIONAL GROUP    3.15000   
23783   2008-09-29   66800  AIG  AMERICAN INTERNATIONAL GROUP    2.50000   
23784   2008-09-30   66800  AIG  AMERICAN INTERNATIONAL GROUP    3.33000   
23785   2008

In [46]:
nans= pd.read_csv('nans.csv')
print(nans['leverage'].describe())
print()
print(nans['equity_volatility'].describe())

count    1922.000000
mean        0.402573
std         0.156279
min         0.008163
25%         0.236821
50%         0.371590
75%         0.579532
max         0.651163
Name: leverage, dtype: float64

count    1922.000000
mean        1.408108
std         0.932102
min         0.148600
25%         0.410367
50%         1.405440
75%         2.013770
max         3.528001
Name: equity_volatility, dtype: float64


In [47]:
print(df['leverage'].describe())
print()
print(df['equity_volatility'].describe())

count    441050.000000
mean          0.263176
std           0.163073
min           0.000000
25%           0.149472
50%           0.240519
75%           0.358133
max           1.039717
Name: leverage, dtype: float64

count    441050.000000
mean          2.226904
std          14.809782
min           0.090416
25%           0.189931
50%           0.243494
75%           0.330184
max         116.571964
Name: equity_volatility, dtype: float64


#### 1% Change in Interest Rate

In [48]:
df= pd.read_csv('../data/clean_data.csv')
df['merton_pd'] = df.apply(
    lambda row: compute_merton_pd(
        E=row['market_cap'],
        sigma_E=row['equity_volatility'],
        D=row['total_debt'],
        r=(row['rf'] / 100) * .99,  # Convert percentage to decimal
        verbose=True
    ),
    axis=1
)
print(df)

/var/folders/59/hb0z0gw54v346g6175w3ns9r0000gp/T/ipykernel_23568/2908855909.py:7: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (np.log(V / D) + (r + 0.5 * sigma_V**2) * T) / (sigma_V * np.sqrt(T))
/var/folders/59/hb0z0gw54v346g6175w3ns9r0000gp/T/ipykernel_23568/2908855909.py:19: RuntimeWarning: divide by zero encountered in scalar divide
  d2 = (np.log(V_opt / D) + (r - 0.5 * sigma_V_opt ** 2) * T) / (sigma_V_opt * np.sqrt(T))


Failure for firm: 12798849840.0 1.184655266718994 133929500000.0 0.034353
Message: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
Failure for firm: 10083127500.0 1.2063649383315511 133929500000.0 0.034451999999999997
Message: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
Failure for firm: 5512109699.999999 1.3433913552168046 133929500000.0 0.033759
Message: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
Failure for firm: 7232963460.0 1.3732017429642132 133929500000.0 0.035046
Message: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
Failure for firm: 10352010900.0 1.4225119557038712 133929500000.0 0.037422
Message: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
Failure for firm: 12691296479.9

In [49]:
df['pd_valid'] = df['merton_pd'].notna()
df.to_csv("merton_model_output2.csv", index=False)

In [50]:
df['pd_valid'].value_counts()

pd_valid
True     439016
False      2034
Name: count, dtype: int64

In [51]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

print(df[df['merton_pd'].isna() == True])
(df[df['merton_pd'].isna() == True]).to_csv("nans.csv", index=False)

pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.max_colwidth')

              date  permno  tic                          conm        PRC  \
23773   2008-09-15   66800  AIG  AMERICAN INTERNATIONAL GROUP    4.76000   
23774   2008-09-16   66800  AIG  AMERICAN INTERNATIONAL GROUP    3.75000   
23775   2008-09-17   66800  AIG  AMERICAN INTERNATIONAL GROUP    2.05000   
23776   2008-09-18   66800  AIG  AMERICAN INTERNATIONAL GROUP    2.69000   
23777   2008-09-19   66800  AIG  AMERICAN INTERNATIONAL GROUP    3.85000   
23778   2008-09-22   66800  AIG  AMERICAN INTERNATIONAL GROUP    4.72000   
23779   2008-09-23   66800  AIG  AMERICAN INTERNATIONAL GROUP    5.00000   
23780   2008-09-24   66800  AIG  AMERICAN INTERNATIONAL GROUP    3.31000   
23781   2008-09-25   66800  AIG  AMERICAN INTERNATIONAL GROUP    3.02000   
23782   2008-09-26   66800  AIG  AMERICAN INTERNATIONAL GROUP    3.15000   
23783   2008-09-29   66800  AIG  AMERICAN INTERNATIONAL GROUP    2.50000   
23784   2008-09-30   66800  AIG  AMERICAN INTERNATIONAL GROUP    3.33000   
23785   2008

In [52]:
nans= pd.read_csv('nans.csv')
print(nans['leverage'].describe())
print()
print(nans['equity_volatility'].describe())

count    2034.000000
mean        0.408919
std         0.153865
min         0.141933
25%         0.255030
50%         0.376105
75%         0.567463
max         0.651163
Name: leverage, dtype: float64

count    2034.000000
mean        1.260902
std         0.896850
min         0.185193
25%         0.339702
50%         1.283351
75%         1.605010
max         3.528001
Name: equity_volatility, dtype: float64


In [53]:
print(df['leverage'].describe())
print()
print(df['equity_volatility'].describe())

count    441050.000000
mean          0.263176
std           0.163073
min           0.000000
25%           0.149472
50%           0.240519
75%           0.358133
max           1.039717
Name: leverage, dtype: float64

count    441050.000000
mean          2.226904
std          14.809782
min           0.090416
25%           0.189931
50%           0.243494
75%           0.330184
max         116.571964
Name: equity_volatility, dtype: float64


#### 10% Change in Debt and Volatility

In [54]:
df= pd.read_csv('../data/clean_data.csv')
df['merton_pd'] = df.apply(
    lambda row: compute_merton_pd(
        E=row['market_cap'],
        sigma_E=row['equity_volatility'] * .9,
        D=row['total_debt'] * .9,
        r=row['rf'] / 100,  # Convert percentage to decimal
        verbose=True
    ),
    axis=1
)
print(df)

/var/folders/59/hb0z0gw54v346g6175w3ns9r0000gp/T/ipykernel_23568/2908855909.py:7: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (np.log(V / D) + (r + 0.5 * sigma_V**2) * T) / (sigma_V * np.sqrt(T))
/var/folders/59/hb0z0gw54v346g6175w3ns9r0000gp/T/ipykernel_23568/2908855909.py:19: RuntimeWarning: divide by zero encountered in scalar divide
  d2 = (np.log(V_opt / D) + (r - 0.5 * sigma_V_opt ** 2) * T) / (sigma_V_opt * np.sqrt(T))


Failure for firm: 12798849840.0 1.0661897400470945 120536550000.0 0.0347
Message: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
Failure for firm: 10083127500.0 1.0857284444983961 120536550000.0 0.0348
Message: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
Failure for firm: 5512109699.999999 1.209052219695124 120536550000.0 0.0341
Message: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
Failure for firm: 7232963460.0 1.235881568667792 120536550000.0 0.0354
Message: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
Failure for firm: 10352010900.0 1.280260760133484 120536550000.0 0.0378
Message: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
Failure for firm: 12691296479.999998 1.2947678932640423

In [57]:
df['pd_valid'] = df['merton_pd'].notna()
df.to_csv("merton_model_output3.csv", index=False)

In [58]:
df['pd_valid'].value_counts()

pd_valid
True     439149
False      1901
Name: count, dtype: int64

In [59]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

print(df[df['merton_pd'].isna() == True])
(df[df['merton_pd'].isna() == True]).to_csv("nans.csv", index=False)

pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.max_colwidth')

              date  permno  tic                          conm        PRC  \
23773   2008-09-15   66800  AIG  AMERICAN INTERNATIONAL GROUP    4.76000   
23774   2008-09-16   66800  AIG  AMERICAN INTERNATIONAL GROUP    3.75000   
23775   2008-09-17   66800  AIG  AMERICAN INTERNATIONAL GROUP    2.05000   
23776   2008-09-18   66800  AIG  AMERICAN INTERNATIONAL GROUP    2.69000   
23777   2008-09-19   66800  AIG  AMERICAN INTERNATIONAL GROUP    3.85000   
23778   2008-09-22   66800  AIG  AMERICAN INTERNATIONAL GROUP    4.72000   
23779   2008-09-23   66800  AIG  AMERICAN INTERNATIONAL GROUP    5.00000   
23780   2008-09-24   66800  AIG  AMERICAN INTERNATIONAL GROUP    3.31000   
23781   2008-09-25   66800  AIG  AMERICAN INTERNATIONAL GROUP    3.02000   
23782   2008-09-26   66800  AIG  AMERICAN INTERNATIONAL GROUP    3.15000   
23783   2008-09-29   66800  AIG  AMERICAN INTERNATIONAL GROUP    2.50000   
23784   2008-09-30   66800  AIG  AMERICAN INTERNATIONAL GROUP    3.33000   
23785   2008

In [60]:
nans= pd.read_csv('nans.csv')
print(nans['leverage'].describe())
print()
print(nans['equity_volatility'].describe())

count    1901.000000
mean        0.419026
std         0.158512
min         0.141933
25%         0.236821
50%         0.377560
75%         0.579532
max         0.651163
Name: leverage, dtype: float64

count    1901.000000
mean        1.338751
std         0.949275
min         0.170934
25%         0.368090
50%         1.353303
75%         1.765070
max         3.528001
Name: equity_volatility, dtype: float64


In [ ]:
print(df['leverage'].describe())
print()
print(df['equity_volatility'].describe())

#### Calculating Deltas